# Project 6: Control Theory Code

Motors in Port A and Port B
Distance Sensor in Port C

## 1. General Check Connection Code

To run a few times to make sure connection is solid and getting good data for battery and Distance Sensor.

In [ ]:
import time
# import the serial communication module (for talking to SPIKE Prime)
from SPIKEPrimeSerial.Serial import SPIKEPrimeSerial as SPIKE

mySPIKE = SPIKE()
print(mySPIKE.ListDevices())
# open a Serial Connection to your SPIKE Prime
name = mySPIKE.OpenSerial()
print('Connected to:',name)

val = mySPIKE.GetValue('hub.battery.voltage()')
print('SPIKE Prime Voltage:',val,'millivolts')

print('Beeeeeep!')
mySPIKE.SendCommand('hub.sound.beep()')

mySPIKE.SendCommand('hub.port.C.device.mode(0)')
time.sleep(1)
val = mySPIKE.GetValue('hub.port.C.device.get()')
print('Distance:',val,'cm')

mySPIKE.CloseSerial()
print('Closed Connection')

## 2. Drive to object

Drive forward till reach object (distance <= 4cm)

In [ ]:
import time
# import the serial communication module (for talking to SPIKE Prime)
from SPIKEPrimeSerial.Serial import SPIKEPrimeSerial as SPIKE

mySPIKE = SPIKE()
print(mySPIKE.ListDevices())
# open a Serial Connection to your SPIKE Prime
name = mySPIKE.OpenSerial()
print('Connected to:',name)

In [ ]:
# this is the function that does the averaging
def avg_sensor(mySPIKE, port, modes):
    time_delay = 0.08 # how long to wait
    average = 0
    num_data_points = 0
    for mode in modes:
        # code that sends the new mode value to the right port
        mySPIKE.SendCommand('hub.port.' + str(port) + '.device.mode(' + str(mode) + ')')
        time.sleep(time_delay)
        val = mySPIKE.GetValue('hub.port.' + str(port) + '.device.get()')
        if type(val) == list:
            sensor_data = val[0]
            if type(sensor_data) == int:
                average = average + sensor_data
                num_data_points = num_data_points + 1
        #time.sleep(time_delay)
    if num_data_points > 0:
        return average/num_data_points
    else:
        return -1 # didn't get any valid data, return -1

In [ ]:
# STOP ROBOT
def STOP(mySPIKE):
    # force motors to stop!
    mySPIKE.SendCommand('hub.display.show("S")')
    mySPIKE.SendCommand('hub.port.A.motor.brake()')
    mySPIKE.SendCommand('hub.port.B.motor.brake()')
    mySPIKE.SendCommand('hub.port.A.motor.float()')
    mySPIKE.SendCommand('hub.port.B.motor.float()')
    return
STOP(mySPIKE)

In [ ]:
# helper function for making sure value is in range
def force_range(x, min_val, max_val):
    x = min(x, max_val)
    x = max(min_val, x)
    return x

def drive(mySPIKE, A=50, B=50):
    # MY HARDWARE: A needs to be "reversed" in order to drive forward
    A = A*-1
    # check to make sure between -100 and 100
    A = force_range(A, -100, 100)
    B = force_range(B, -100, 100)
    # run (start) motors:
    mySPIKE.SendCommand('hub.port.A.motor.pwm(' + str(A) + ')')
    mySPIKE.SendCommand('hub.port.B.motor.pwm(' + str(B) + ')')
    return

In [ ]:
# test that the driving works
drive(mySPIKE, A=50, B=50) # half speed
time.sleep(2) # for two seconds
drive(mySPIKE, A=25, B=25) # quarter speed
time.sleep(2) # for two seconds
STOP(mySPIKE)

In [ ]:
# 2. WRITE CODE HERE TO DRIVE TO OBJECT
current_val = avg_sensor(mySPIKE, 'C', [0]) # try mode 0
print(current_val)
while current_val > 5:
    drive(mySPIKE, A=50, B=50)
    current_val = avg_sensor(mySPIKE, 'C', [0]) # try mode 0
    print(current_val)
STOP(mySPIKE)

In [ ]:
mySPIKE.CloseSerial()
print('Closed Connection')

## 3. Drive proportionally to object

In [ ]:
# 3. WRITE CODE HERE TO DRIVE PROPORTIONALLY TO OBJECT
current_val = avg_sensor(mySPIKE, 'C', [0]) # try mode 0
print(current_val)
while current_val > 5:
    current_val = avg_sensor(mySPIKE, 'C', [0]) # try mode 0
    print(current_val)
    drive(mySPIKE, A=int(current_val+18), B=int(current_val+18))
STOP(mySPIKE)

## 4. Drive faster to object

In [ ]:
# 4. WRITE CODE HERE TO DRIVE FASTER TO OBJECT
Kp = 2 # proportionality constant Kp
current_val = avg_sensor(mySPIKE, 'C', [0]) # try modes 0
print(current_val)
while current_val > 5:
    current_val = avg_sensor(mySPIKE, 'C', [0]) # try modes 0
    print(current_val)
    drive(mySPIKE, A=Kp*int(current_val)+8, B=Kp*int(current_val)+8)
STOP(mySPIKE)

## 5. Drive to distance away from object

In [ ]:
# 5. WRITE CODE HERE TO DRIVE DISTANCE AWAY FROM OBJECT
Kp = 2
goal = 33
current_val = avg_sensor(mySPIKE, 'C', [0]) # try mode 0
error = current_val - goal # error is how far away are we from our goal
print(current_val, 'Error:', error)
while abs(error) > 7:
    current_val = avg_sensor(mySPIKE, 'C', [0]) # try mode 0
    error = current_val - goal # error is how far away are we from our goal
    print(current_val, 'Error:', error)
    drive(mySPIKE, A=Kp*int(error), B=Kp*int(error))
STOP(mySPIKE)

## 6. Stay certain distance from object

Same code works if before goal or after goal!

In [ ]:
# 6. WRITE CODE HERE TO STAY CERTAIN DISTANCE AWAY FROM OBJECT
Kp = 7
goal = 33
current_val = avg_sensor(mySPIKE, 'C', [0]) # try modes 0, 1, and 2
error = current_val - goal # error is how far away are we from our goal
print(current_val, 'Error:', error)
while True:
    current_val = avg_sensor(mySPIKE, 'C', [0]) # try modes 0, 1, and 2
    error = current_val - goal # error is how far away are we from our goal
    print(current_val, 'Error:', error)
    drive(mySPIKE, A=Kp*int(error), B=Kp*int(error))
STOP(mySPIKE)

## 7. Rotate to Angle

In [ ]:
# here is helper code for position
def getPosition(mySPIKE, direction='X'):
    val = mySPIKE.GetValue('hub.motion.position()')
    if type(val) == tuple:
        position_x = val[0] # convert value to integer
        position_y = val[1] # convert value to integer
        position_z = val[2] # convert value to integer
    else:
        position_x = 'Error (' + str(val) + ')' # set to string
        position_y = 'Error (' + str(val) + ')' # set to string
        position_z = 'Error (' + str(val) + ')' # set to string
    if direction == 'X':
        return position_x
    elif direction == 'Y':
        return position_y
    else:
        return position_z

In [ ]:
# Test getting position to make sure reading it right
current_val = getPosition(mySPIKE, direction='X')
print(current_val)

In [ ]:
# 7. WRITE CODE HERE TO ROTATE TO SPECIFIC ANGLE
orig_val = getPosition(mySPIKE, direction='X') # record where we are starting
print('Starting location:', orig_val)
Kp = 2
goal = 90
current_val = (getPosition(mySPIKE, direction='X') - orig_val)
error = current_val - goal
print(current_val, 'Error:', error)
while True:
    current_val = (getPosition(mySPIKE, direction='X') - orig_val)
    error = current_val - goal
    print(current_val, 'Error:', error)
    # throw in "-1" times A value to turn in circle
    drive(mySPIKE, A=-1*Kp*int(error), B=Kp*int(error))
STOP(mySPIKE)

In [ ]:
# STOP ROBOT
STOP(mySPIKE)

## Dr. E's SCRATCH CODE

Down here is code I was using as I tested this out before class.

In [ ]:
# 2. WRITE CODE HERE TO GO TO DRIVE TO OBJECT
current_val = avg_sensor(mySPIKE, 'C', [0, 1, 2]) # try modes 0, 1, and 2
print(current_val)
while current_val > 4:
    current_val = avg_sensor(mySPIKE, 'C', [0, 1, 2]) # try modes 0, 1, and 2
    print(current_val)
    drive(mySPIKE, A=50, B=50)
STOP(mySPIKE)

In [ ]:
# 3. WRITE CODE HERE TO DRIVE PROPORTIONALLY TO OBJECT
current_val = avg_sensor(mySPIKE, 'C', [0, 1, 2]) # try modes 0, 1, and 2
print(current_val)
while current_val > 4:
    current_val = avg_sensor(mySPIKE, 'C', [0, 1, 2]) # try modes 0, 1, and 2
    print(current_val)
    drive(mySPIKE, A=int(current_val), B=int(current_val))
STOP(mySPIKE)

In [ ]:
def in_range(x, min_val=-100, max_val=100):
    x = max(x, min_val)
    x = min(x, max_val)
    return x

In [ ]:
# 4. WRITE CODE HERE TO DRIVE FASTER TO OBJECT
Kp = 7
current_val = avg_sensor(mySPIKE, 'C', [0, 1, 2]) # try modes 0, 1, and 2
print(current_val)
while current_val > 4:
    current_val = avg_sensor(mySPIKE, 'C', [0, 1, 2]) # try modes 0, 1, and 2
    print(current_val)
    drive(mySPIKE, A=in_range((Kp*int(current_val))), B=in_range(Kp*int(current_val)))
STOP(mySPIKE)

In [ ]:
# 5. WRITE CODE HERE TO DRIVE DISTANCE AWAY FROM OBJECT
Kp = 2
goal = 30
current_val = avg_sensor(mySPIKE, 'C', [0, 1, 2]) # try modes 0, 1, and 2
error = current_val - goal
print(current_val,'Error:',error)
while error > 0:
    current_val = avg_sensor(mySPIKE, 'C', [0, 1, 2]) # try modes 0, 1, and 2
    error = current_val - goal
    print(current_val,'Error:',error)
    drive(mySPIKE, A=in_range((Kp*int(error))), B=in_range(Kp*int(error)))
STOP(mySPIKE)

In [ ]:
# 6. WRITE CODE HERE TO STAY CERTAIN DISTANCE AWAY FROM OBJECT
Kp = 3
goal = 30
current_val = avg_sensor(mySPIKE, 'C', [0, 1, 2]) # try modes 0, 1, and 2
error = current_val - goal
print(current_val,'Error:',error)
while True:
    current_val = avg_sensor(mySPIKE, 'C', [0, 1, 2]) # try modes 0, 1, and 2
    error = current_val - goal
    print(current_val,'Error:',error)
    drive(mySPIKE, A=in_range((Kp*int(error))), B=in_range(Kp*int(error)))
STOP(mySPIKE)

In [ ]:
# 7. WRITE CODE HERE TO ROTATE TO SPECIFIC ANGLE
Kp = 1
goal = 90 # degrees
current_val = getPosition(mySPIKE, direction='X')
error = current_val - goal
print(current_val,'Error:',error)
while True:
    current_val = getPosition(mySPIKE, direction='X')
    error = current_val - goal
    print(current_val,'Error:',error)
    drive(mySPIKE, A=-1*in_range((Kp*int(error))), B=in_range(Kp*int(error)))
STOP(mySPIKE)